# Multiple knapsacks

## Intro

This is a generalization of the knapsack problem, in which the goal is to carry the maximum value in a single backpack considering a given set of items and a maximum capacity.

## Model statement

$$
\begin{align}
    \text{max} \quad & \sum_{i \in I}{c_{i} x_{i, j}} \\
    \text{s.t.} \quad & \sum_{i \in I}{w_{d, i} x_{i, j}} \leq k_{d, j} & \forall ~d \in D; j \in J\\
    & \sum_{j \in J}{x_{i, j}} \leq b_{i} & \forall ~ i \in I\\
    & x_i \in \Z & \forall ~ i \in I\\
\end{align}
$$

In [59]:
import pandas as pd
import pyomo.environ as pyo

In [60]:
# Here it is the information about items available
data_items = pd.read_excel("../data/multi_knapsacks.xlsx", sheet_name=0, index_col=0)

# And here about the knapsacks
data_knapsacks = pd.read_excel("../data/multi_knapsacks.xlsx", sheet_name=1, index_col=0)

In [61]:
data_items.head()

,weight,volume,value,available
1,4.661295,1.002111,1.947939,1
2,7.514717,1.146791,5.189848,1
3,8.996028,5.010643,4.431570,1
4,3.684529,4.489547,6.157468,2
5,5.811656,3.572954,5.374311,1


In [62]:
data_knapsacks.head()

,weight,volume
A,21,32
B,19,22
C,28,17


In [63]:
# Create sets of unique values of the sets
dimensions = data_knapsacks.columns.to_numpy()
items_labels = data_items.index.unique().to_numpy()
knapsacks = data_knapsacks.index.unique().to_numpy()

## Using pyomo

In [64]:
model = pyo.ConcreteModel()

### Sets

In [65]:
model.D = pyo.Set(initialize=dimensions)
model.I = pyo.Set(initialize=items_labels)
model.J = pyo.Set(initialize=knapsacks)

### Parameters

In [66]:
k = data_knapsacks.melt(ignore_index=False)\
    .set_index(keys=["variable"], append=True)\
        .swaplevel(0, 1).to_dict()["value"]
model.k = pyo.Param(model.D, model.J, initialize=k)

In [67]:
w = data_items[dimensions].melt(ignore_index=False)\
    .set_index(keys=["variable"], append=True)\
        .swaplevel(0, 1).to_dict()["value"]
items_available = data_items["available"].to_dict()
items_values = data_items["value"].to_dict()

model.b = pyo.Param(model.I, initialize=items_available)
model.w = pyo.Param(model.D, model.I, initialize=w)
model.c = pyo.Param(model.I, initialize=items_values)

### Decision variables

In [68]:
bounds_n_items = {}

for key, value in items_available.items():
    bounds_n_items[key] = (0, value)

def get_bounds(model, i, j):
    return bounds_n_items[i]

In [69]:
model.x = pyo.Var(model.I, model.J, within=pyo.Integers, bounds=get_bounds)

### Constraints

In [70]:
def availability_constraint(model, i):
    return sum(model.x[i, j] for j in model.J) <= model.b[i]

model.availability_constraint = pyo.Constraint(model.I, rule=availability_constraint)


def capacity_constraint(model, d, j):
    return sum(model.x[i, j] * model.w[d, i] for i in model.I) <= model.k[d, j]

model.capacity_constraint = pyo.Constraint(model.D, model.J, rule=capacity_constraint)

### Objective

In [71]:
def obj_function(model):
    return sum(model.x[i, j] * model.c[i]\
        for i in model.I for j in model.J)
    
model.objective = pyo.Objective(rule=obj_function, sense=pyo.maximize)

### Solve

In [72]:
# model.write("model.mps", io_options={'symbolic_solver_labels': True})

In [73]:
opt = pyo.SolverFactory('cbc')
opt.options['sec'] = 120
opt.options['cuts'] = 'on'
opt.options['heur'] = 'on'

In [74]:
solution = opt.solve(model)
print(solution)

    containing a solution

Problem: 
- Name: unknown
  Lower bound: 127.73843799
  Upper bound: -127.78033
  Number of objectives: 1
  Number of constraints: 56
  Number of variables: 150
  Number of binary variables: 99
  Number of integer variables: 150
  Number of nonzeros: 150
  Sense: maximize
Solver: 
- Status: aborted
  User time: -1.0
  System time: 120.13
  Wallclock time: 120.13
  Termination condition: maxTimeLimit
  Termination message: Optimization terminated because the time expended exceeded the value specified in the seconds parameter.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 748890
      Number of created subproblems: 748890
    Black box: 
      Number of iterations: 3062360
  Error rc: 0
  Time: 120.42014813423157
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [75]:
model.objective()

127.738437990416

In [76]:
model.capacity_constraint.display()

capacity_constraint : Size=6
    Key             : Lower : Body               : Upper
    ('volume', 'A') :  None : 31.390966375697978 :  32.0
    ('volume', 'B') :  None : 21.194491520099053 :  22.0
    ('volume', 'C') :  None : 16.767874386429273 :  17.0
    ('weight', 'A') :  None : 20.983448820880767 :  21.0
    ('weight', 'B') :  None : 18.982435077352662 :  19.0
    ('weight', 'C') :  None :  27.99791623266966 :  28.0


In [77]:
output = pd.DataFrame(index=items_labels, columns=knapsacks)

for i in items_labels:
    for k in knapsacks:
        output.loc[i, k] = model.x[i, k].value

output.to_excel("../data/output_multiple_knapsacks.xlsx")